# Сравнение оптимизаций обучения

В этом ноутбуке мы сравним производительность трёх конфигураций обучения:

1. **FP32 + Baseline DataLoader** - без оптимизаций
2. **FP32 + Optimized DataLoader** - с `pin_memory=True` и `non_blocking=True`
3. **FP16 + Optimized DataLoader** - mixed precision с оптимизированным DataLoader

Для каждой конфигурации мы запустим профилировщик и выведем результаты.

**Параметры:** 10 эпох обучения для каждого эксперимента.


In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.profilers import SimpleProfiler
import time

from text_features_data import TextFeaturesDataModule
from mlp_model import DeepMLP
from lightning_module import TextClassificationModule
from optimized_strategy import OptimizedSingleDeviceStrategy

# Проверяем наличие GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


Using device: cuda
GPU: NVIDIA L40
GPU memory: 44.38 GB


## Эксперимент 1: FP32 + Baseline DataLoader

Обучение без оптимизаций - baseline для сравнения.


In [2]:
# DataModule без оптимизаций
dm_baseline = TextFeaturesDataModule(
    max_features=20000,
    batch_size=256,
    use_bigrams=True,
    cache_dir="./cache",
    num_workers=4,
    pin_memory=False,  # Без pin_memory
)

dm_baseline.prepare_data()
dm_baseline.setup()

print(f"\nDataset info:")
print(f"  Input dim: {dm_baseline.input_dim}")
print(f"  Num classes: {dm_baseline.n_classes}")
print(f"  Train samples: {len(dm_baseline.train_dataset)}")
print(f"  Val samples: {len(dm_baseline.val_dataset)}")


Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140

Dataset info:
  Input dim: 20005
  Num classes: 4
  Train samples: 108460
  Val samples: 19140


In [3]:
# Создаем модель
model_1 = DeepMLP(
    input_dim=dm_baseline.input_dim,
    output_dim=dm_baseline.n_classes,
    hidden_dims=[2048, 1024, 512, 256, 128],
    activation='silu',
    dropout=0.3,
    use_batch_norm=True,
)

lightning_model_1 = TextClassificationModule(
    model=model_1,
    lr=1e-3,
    optimizer_type='adam',
    weight_decay=1e-5,
    num_classes=dm_baseline.n_classes,
)

# Профилировщик
profiler_1 = SimpleProfiler(dirpath='./profiler', filename='exp1_fp32_baseline')

# Trainer
trainer_1 = pl.Trainer(
    max_epochs=10,  # 10 эпох
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    profiler=profiler_1,
    enable_progress_bar=True,
)

print("\n" + "="*80)
print("ЭКСПЕРИМЕНТ 1: FP32 + Baseline DataLoader")
print("="*80)

# Обучение с замером времени
torch.cuda.synchronize() if torch.cuda.is_available() else None
start_time = time.time()

trainer_1.fit(lightning_model_1, dm_baseline)

torch.cuda.synchronize() if torch.cuda.is_available() else None
training_time_1 = time.time() - start_time

print(f"\n✓ Обучение завершено за {training_time_1:.2f} секунд ({training_time_1/60:.2f} минут)")

# Очистка памяти
del model_1, lightning_model_1, trainer_1
torch.cuda.empty_cache() if torch.cuda.is_available() else None


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



ЭКСПЕРИМЕНТ 1: FP32 + Baseline DataLoader


You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | DeepMLP            | 43.8 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | val_f1    | MulticlassF1Score  | 0      | train
---------------------------------------------------------
43.8 M    Trainable params
0         Non-trainable params
43.8 M    Total params
175.072   Total estimated model params size (MB)
23        Modules in train mode
0         Modu

Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.



✓ Обучение завершено за 120.96 секунд (2.02 минут)


In [4]:
# Результаты профилировщика
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 1")
print("="*80 + "\n")
print(profiler_1.summary())



РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 1

FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                      

## Эксперимент 2: FP32 + Optimized DataLoader

Обучение с оптимизированным DataLoader:
- `pin_memory=True` - ускоряет передачу данных CPU → GPU
- `non_blocking=True` в стратегии - асинхронная передача данных


In [5]:
# DataModule с оптимизациями
dm_optimized = TextFeaturesDataModule(
    max_features=20000,
    batch_size=256,
    use_bigrams=True,
    cache_dir="./cache",
    num_workers=4,
    pin_memory=True,  # Оптимизация: pin_memory
)

dm_optimized.prepare_data()
dm_optimized.setup()

print(f"\nDataset info:")
print(f"  Input dim: {dm_optimized.input_dim}")
print(f"  pin_memory: {dm_optimized.pin_memory}")


Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140

Dataset info:
  Input dim: 20005
  pin_memory: True


In [6]:
# Создаем модель
model_2 = DeepMLP(
    input_dim=dm_optimized.input_dim,
    output_dim=dm_optimized.n_classes,
    hidden_dims=[2048, 1024, 512, 256, 128],
    activation='silu',
    dropout=0.3,
    use_batch_norm=True,
)

lightning_model_2 = TextClassificationModule(
    model=model_2,
    lr=1e-3,
    optimizer_type='adam',
    weight_decay=1e-5,
    num_classes=dm_optimized.n_classes,
)

# Профилировщик
profiler_2 = SimpleProfiler(dirpath='./profiler', filename='exp2_fp32_optimized')

# Оптимизированная стратегия с non_blocking
strategy_2 = OptimizedSingleDeviceStrategy(device='cuda:0' if torch.cuda.is_available() else 'cpu')

# Trainer
trainer_2 = pl.Trainer(
    max_epochs=10,  # 10 эпох
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    strategy=strategy_2,
    profiler=profiler_2,
    enable_progress_bar=True,
)

print("\n" + "="*80)
print("ЭКСПЕРИМЕНТ 2: FP32 + Optimized DataLoader")
print("="*80)

# Обучение с замером времени
torch.cuda.synchronize() if torch.cuda.is_available() else None
start_time = time.time()

trainer_2.fit(lightning_model_2, dm_optimized)

torch.cuda.synchronize() if torch.cuda.is_available() else None
training_time_2 = time.time() - start_time

print(f"\n✓ Обучение завершено за {training_time_2:.2f} секунд ({training_time_2/60:.2f} минут)")
print(f"  Ускорение относительно baseline: {training_time_1/training_time_2:.2f}x")

# Очистка памяти
del model_2, lightning_model_2, trainer_2
torch.cuda.empty_cache() if torch.cuda.is_available() else None


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



ЭКСПЕРИМЕНТ 2: FP32 + Optimized DataLoader


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | DeepMLP            | 43.8 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | val_f1    | MulticlassF1Score  | 0      | train
---------------------------------------------------------
43.8 M    Trainable params
0         Non-trainable params
43.8 M    Total params
175.072   Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.



✓ Обучение завершено за 89.44 секунд (1.49 минут)
  Ускорение относительно baseline: 1.35x


In [7]:
# Результаты профилировщика
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 2")
print("="*80 + "\n")
print(profiler_2.summary())



РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 2

FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                      

## Эксперимент 3: FP16 + Optimized DataLoader

Обучение с mixed precision (FP16) и оптимизированным DataLoader:
- `precision='16-mixed'` - автоматическое mixed precision
- `pin_memory=True` - ускорение передачи данных
- `non_blocking=True` - асинхронная передача


In [8]:
# Используем тот же оптимизированный DataModule
print(f"\nИспользуем оптимизированный DataModule:")
print(f"  pin_memory: {dm_optimized.pin_memory}")



Используем оптимизированный DataModule:
  pin_memory: True


In [9]:
# Создаем модель
model_3 = DeepMLP(
    input_dim=dm_optimized.input_dim,
    output_dim=dm_optimized.n_classes,
    hidden_dims=[2048, 1024, 512, 256, 128],
    activation='silu',
    dropout=0.3,
    use_batch_norm=True,
)

lightning_model_3 = TextClassificationModule(
    model=model_3,
    lr=1e-3,
    optimizer_type='adam',
    weight_decay=1e-5,
    num_classes=dm_optimized.n_classes,
)

# Профилировщик
profiler_3 = SimpleProfiler(dirpath='./profiler', filename='exp3_fp16_optimized')

# Оптимизированная стратегия с non_blocking
strategy_3 = OptimizedSingleDeviceStrategy(device='cuda:0' if torch.cuda.is_available() else 'cpu')

# Trainer с FP16
trainer_3 = pl.Trainer(
    max_epochs=10,  # 10 эпох
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision='16-mixed',  # FP16 mixed precision
    strategy=strategy_3,
    profiler=profiler_3,
    enable_progress_bar=True,
)

print("\n" + "="*80)
print("ЭКСПЕРИМЕНТ 3: FP16 + Optimized DataLoader")
print("="*80)

# Обучение с замером времени
torch.cuda.synchronize() if torch.cuda.is_available() else None
start_time = time.time()

trainer_3.fit(lightning_model_3, dm_optimized)

torch.cuda.synchronize() if torch.cuda.is_available() else None
training_time_3 = time.time() - start_time

print(f"\n✓ Обучение завершено за {training_time_3:.2f} секунд ({training_time_3/60:.2f} минут)")
print(f"  Ускорение относительно baseline: {training_time_1/training_time_3:.2f}x")
print(f"  Ускорение относительно FP32 optimized: {training_time_2/training_time_3:.2f}x")

# Очистка памяти
del model_3, lightning_model_3, trainer_3
torch.cuda.empty_cache() if torch.cuda.is_available() else None


Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



ЭКСПЕРИМЕНТ 3: FP16 + Optimized DataLoader


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | DeepMLP            | 43.8 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | val_f1    | MulticlassF1Score  | 0      | train
---------------------------------------------------------
43.8 M    Trainable params
0         Non-trainable params
43.8 M    Total params
175.072   Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.



✓ Обучение завершено за 95.22 секунд (1.59 минут)
  Ускорение относительно baseline: 1.27x
  Ускорение относительно FP32 optimized: 0.94x


In [10]:
# Результаты профилировщика
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 3")
print("="*80 + "\n")
print(profiler_3.summary())



РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 3

FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                      

## Итоговое сравнение

Сравним время обучения всех трёх конфигураций.


In [11]:
print("\n" + "="*80)
print("ИТОГОВОЕ СРАВНЕНИЕ ВРЕМЕНИ ОБУЧЕНИЯ (10 ЭПОХ)")
print("="*80 + "\n")

print(f"Эксперимент 1 (FP32 Baseline):          {training_time_1:6.2f}s ({training_time_1/60:5.2f}m) - базовая линия")
print(f"Эксперимент 2 (FP32 Optimized):         {training_time_2:6.2f}s ({training_time_2/60:5.2f}m) - ускорение {training_time_1/training_time_2:.2f}x")
print(f"Эксперимент 3 (FP16 + Optimized):       {training_time_3:6.2f}s ({training_time_3/60:5.2f}m) - ускорение {training_time_1/training_time_3:.2f}x")

print("\n" + "="*80)
print("ВЫВОДЫ:")
print("="*80)
print(f"1. Оптимизация DataLoader (pin_memory + non_blocking) дала ускорение {training_time_1/training_time_2:.2f}x")
print(f"2. FP16 mixed precision дала дополнительное ускорение {training_time_2/training_time_3:.2f}x")
print(f"3. Общее ускорение от всех оптимизаций: {training_time_1/training_time_3:.2f}x")
print(f"4. Экономия времени: {training_time_1 - training_time_3:.1f}s ({(training_time_1 - training_time_3)/60:.2f}m) на 10 эпох")
print("="*80)



ИТОГОВОЕ СРАВНЕНИЕ ВРЕМЕНИ ОБУЧЕНИЯ (10 ЭПОХ)

Эксперимент 1 (FP32 Baseline):          120.96s ( 2.02m) - базовая линия
Эксперимент 2 (FP32 Optimized):          89.44s ( 1.49m) - ускорение 1.35x
Эксперимент 3 (FP16 + Optimized):        95.22s ( 1.59m) - ускорение 1.27x

ВЫВОДЫ:
1. Оптимизация DataLoader (pin_memory + non_blocking) дала ускорение 1.35x
2. FP16 mixed precision дала дополнительное ускорение 0.94x
3. Общее ускорение от всех оптимизаций: 1.27x
4. Экономия времени: 25.7s (0.43m) на 10 эпох
